# Practica 4

In [2]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

In [3]:
weights = loadmat('ex4weights.mat')
theta1, theta2 = weights['Theta1'], weights['Theta2']

data = loadmat('ex4data1.mat')
y = data['y']
X = data['X']

In [4]:
y = np.ravel(y)
m=len(y)
X_1 = np.hstack([np.ones([m, 1]), X])

In [6]:
def sigmoide(z):
    return (1.0/(1.0+np.exp(-z)))

In [7]:
a_1 = X_1;
z_2 = np.matmul(a_1, theta1.T)
a_2 = sigmoide(z_2)
a_2 = np.hstack([np.ones([len(a_2), 1]), a_2])
z_3 = np.matmul(a_2, theta2.T)
a_3 = sigmoide(z_3)

In [ ]:
def coste(H, y, m):
    